In [1]:
import tensorflow as tf
import os
import numpy as np

### 2.2 Representing tensors###
Tensorflow uses the infamous __tensors__ to represent matrices of any dimensionality. (E.g. a 3D matrix can be represented
as a _rank 3_ tensor in tensorflow). <br>
Tensors cann be created from different sources such as _Python lists, numpy-arrays, tensorflow-constants, etc._ by using the: <br> `tf.convert_to_tensor([source])` <br>
function. `Notice that tf.constant()` also returns a Tensor object:

In [9]:
py_list = [[1.0, 2.0],
           [3.0, 4.0]]

numpy_array = np.array([[1.0, 2.0],
                        [3.0, 4.0]], dtype=np.float32)

tf_constant = tf.constant([[1.0, 2.0],
                           [3.0, 4.0]])

tensor1 = tf.convert_to_tensor(py_list, dtype=tf.float32)
tensor2 = tf.convert_to_tensor(numpy_array, dtype=tf.float32)
tensor3 = tf.convert_to_tensor(tf_constant, dtype=tf.float32)

print(type(py_list))
print(type(numpy_array))
print(type(tensor3))
print(type(tf_constant))
print("----")
print(tensor3)
print(tf_constant)

<class 'list'>
<class 'numpy.ndarray'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
----
Tensor("Const_12:0", shape=(2, 2), dtype=float32)
Tensor("Const_12:0", shape=(2, 2), dtype=float32)


As you can see from the print statements above, each Tensor object has a unique __name__, __shape__, and __dtype__. Because we did not explicitly provide a name, tensorflow automatically generated names for us (e.g. "Const_12_0"). <br><br>
Tensorflow has a few convenient constructors for initialising frequently used tensors such as: <br>
- `tf.zeros(shape)`
- `tf.ones(shape)`

These can be used with standard numerical operations to create other numbers. For example, we can create a 500x500 tensor containing of 0.5s:

In [10]:
halves = tf.ones([500, 500]) * 0.5

In [3]:
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    result = sess.run(neg_x)

print(result)

[-1. -2.]


In [8]:
sess = tf.InteractiveSession()
raw_data = [1., 2., 8., -1., 0., 5.5, 6., 13]
spike = tf.Variable(False)
spike.initializer.run()

In [9]:
for i in range(1, len(raw_data)):
    if raw_data[i] - raw_data[i-1] > 5:
        updater = tf.assign(spike, True)
        updater.eval()
    else:
        tf.assign(spike, False).eval()
    
    print("Spike: %s" % spike.eval())

Spike: False
Spike: True
Spike: False
Spike: False
Spike: True
Spike: False
Spike: True


In [10]:
sess.close()

In [29]:
# Suppose we want to save results so that they are accessible without rerunning the whole model.
# Tensorflow offers us a simple way to do this:

sess = tf.InteractiveSession()
 
raw_data = [1., 2., 8., -1., 0., 5.5, 6., 13]
spikes = tf.Variable([False] * len(raw_data), name='spikes')
spikes.initializer.run()
 
saver = tf.train.Saver()
 
for i in range(1, len(raw_data)):
    if raw_data[i] - raw_data[i-1] > 5:
        spikes_val = spikes.eval()
        spikes_val[i] = True
        updater = tf.assign(spikes, spikes_val)
        updater.eval()
 
 
save_path = saver.save(sess, "./trained_variables.ckpt")
# print("spikes data saved in file: %s" % save_path)
 
sess.close()


FailedPreconditionError: Attempting to use uninitialized value Variable
	 [[Node: save_12/SaveV2 = SaveV2[dtypes=[DT_BOOL, DT_BOOL, DT_BOOL, DT_BOOL, DT_BOOL, DT_BOOL, DT_BOOL, DT_BOOL, DT_BOOL, DT_BOOL, DT_BOOL, DT_BOOL, DT_BOOL, DT_BOOL], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save_12/Const_0, save_12/SaveV2/tensor_names, save_12/SaveV2/shape_and_slices, Variable, spikes, spikes_1, spikes_10, spikes_11, spikes_12, spikes_2, spikes_3, spikes_4, spikes_5, spikes_6, spikes_7, spikes_8, spikes_9)]]

Caused by op 'save_12/SaveV2', defined at:
  File "C:\Users\mrfksiv\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\mrfksiv\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\mrfksiv\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\mrfksiv\Anaconda3\envs\tensorflow\li